In [0]:
from datetime import datetime
notebook_start_datetime=datetime.now()
notebook_start_date=str(notebook_start_datetime)
notebook_start_date=notebook_start_date[0:19]

In [0]:
# %sql
# CREATE SCHEMA IF NOT EXISTS generic;

In [0]:
# %sql
# CREATE OR REPLACE TABLE generic.tbl_generic_testing_results (
#     Test_ID BIGINT GENERATED BY DEFAULT AS IDENTITY,
#     Databricks_View_Name STRING,
#     SAP_View_Name STRING,
#     Test_Name STRING,
#     Test_Description STRING,
#     Test_Start_Date STRING,
#     Test_End_Date STRING,
#     Test_Execution_Time_In_Seconds STRING,
#     Databricks_View_Count STRING,
#     SAP_View_Count STRING,
#     Check_Difference_Count_Between_Databricks_And_SAP STRING,
#     Check_Difference_Sum_Between_Databricks_And_SAP STRING,
#     Test_validation_Status STRING,
#     Comment STRING  
#  )

In [0]:
databricks_view_name = dbutils.widgets.get("databricks_view_name")
SAP_View_Name = dbutils.widgets.get("SAP_View_Name")
databricks_database_name = dbutils.widgets.get("databricks_database_name")
sap_column_count = int(dbutils.widgets.get("sap_column_count"))
sap_row_count = int(dbutils.widgets.get("sap_row_count"))

sap_dup_record_count = int(dbutils.widgets.get("sap_duplicate_record_count"))
sap_col_name = dbutils.widgets.get("sap_null_column_name_list").split(',')
#########################################
sap_col_null_value_count =dbutils.widgets.get("sap_null_column_value_list").split(',')
list1=[]
for x in sap_col_null_value_count:
  list1.append(int(x))
sap_col_null_value_count=list1
############################################
sap_column_name_lst= dbutils.widgets.get("sap_sum_column_name_list").split(',')
############################################
sap_column_sum_lst = dbutils.widgets.get("sap_sum_column_value_list").split(',')
list1=[]
for x in sap_column_sum_lst:
  list1.append(float(x))
sap_column_sum_lst=list1
###############################################
execution_time_of_sap_view=dbutils.widgets.get("execution_time_of_sap_view")

In [0]:
#generating dataframe for original view

def convert_DF (databricks_view_name,databricks_database_name):
  df = spark.sql(f"select * from {databricks_database_name}.{databricks_view_name}")
  return df

In [0]:
#calling function of converting dataframe
try:
    databricks_view_df=convert_DF (databricks_view_name,databricks_database_name)
except:
    start_time=datetime.now()
    start_date=str(start_time)
    start_date=start_date[0:19]
    spark.sql(
            f"""
                INSERT INTO 
                hive_metastore.generic.tbl_generic_testing_results 
                (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                VALUES
                ('{databricks_view_name}','','','Error Found while converting to DataFrame','{start_date}','','','','','','','FAIL','ExceptionRaised: Table or view not found : please check view name and database name')
        """
    )
    dbutils.notebook.exit("ExceptionRaised: Table or view not found : please check view name and database name")

In [0]:
#Row count validation Testing
def row_count_validation(databricks_view_df,sap_row_count):
    start_time=datetime.now()
    start_date=str(start_time)
    start_date=start_date[0:19]
    row_count_databricks_view = databricks_view_df.count()
    row_count_matched = str(row_count_databricks_view == sap_row_count)
    diff_row_count=str(abs(row_count_databricks_view - sap_row_count))
    row_count_databricks_view=str(row_count_databricks_view)
    sap_row_count=str(sap_row_count)
    end_time=datetime.now()
    end_date=str(end_time)
    end_date=end_date[0:19]
    execution_time=str(end_time-start_time)
    execution_time=execution_time[5:]
    
    if row_count_matched =='True':
      spark.sql(
            f"""
                INSERT INTO 
                    hive_metastore.generic.tbl_generic_testing_results 
                    (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                    VALUES
                    ('{databricks_view_name}','{SAP_View_Name}','Row Count','Row count validation of databricks and SAP view','{start_date}','{end_date}','{execution_time}','{row_count_databricks_view}','{sap_row_count}','{diff_row_count}','','PASS','')
            """
        )
    else:
      spark.sql(
            f"""
                INSERT INTO 
                    hive_metastore.generic.tbl_generic_testing_results 
                    (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                    VALUES
                    ('{databricks_view_name}','{SAP_View_Name}','Row Count','Row count validation of databricks and SAP view','{start_date}','{end_date}','{execution_time}','{row_count_databricks_view}','{sap_row_count}','{diff_row_count}','','FAIL','')
            """
        )

    return row_count_matched

In [0]:
row_count_validation=row_count_validation(databricks_view_df,sap_row_count)

In [0]:
#Column Count validation testing

def number_of_columns (databricks_view_df,sap_column_count):
  start_time=datetime.now()
  start_date=str(start_time)
  start_date=start_date[0:19]
  databricks_col_count=len(databricks_view_df.columns)
  difference_col_count=str(abs(databricks_col_count-sap_column_count))
  col_count_val_status=str(databricks_col_count==sap_column_count)
  sap_column_count=str(sap_column_count)
  end_time=datetime.now()
  end_date=str(end_time)
  end_date=end_date[0:19]
  execution_time=str(end_time-start_time)
  execution_time=execution_time[5:]
  
  if col_count_val_status=='True':
    spark.sql(
          f"""
              INSERT INTO 
                  hive_metastore.generic.tbl_generic_testing_results 
                  (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                  VALUES
                  ('{databricks_view_name}','{SAP_View_Name}','Column Count','Column count validation of databricks and SAP view','{start_date}','{end_date}','{execution_time}','{databricks_col_count}','{sap_column_count}','{difference_col_count}','','PASS','')
          """
      )
  else:
    spark.sql(
          f"""
              INSERT INTO 
                  hive_metastore.generic.tbl_generic_testing_results 
                  (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                  VALUES
                  ('{databricks_view_name}','{SAP_View_Name}','Column Count','Column count validation of databricks and SAP view','{start_date}','{end_date}','{execution_time}','{databricks_col_count}','{sap_column_count}','{difference_col_count}','','FAIL','')
          """
      )

  return col_count_val_status

In [0]:
col_count_val_status=number_of_columns(databricks_view_df,sap_column_count)

In [0]:
# Duplicate records count validation testing
import pyspark.sql.functions as f
def Count_duplicate_records(databricks_view_df,sap_dup_record_count):
  start_time=datetime.now()
  start_date=str(start_time)
  start_date=start_date[0:19]
  
  databricks_dup_record_count=(databricks_view_df.groupBy(databricks_view_df.columns).agg((f.count("*")>1).cast("int").alias("dup_cnt")).where(f.col("dup_cnt") > 1)).count()


  diff_dup_count=str(abs(databricks_dup_record_count-sap_dup_record_count))
  duplicate_record_val_status=str(databricks_dup_record_count==sap_dup_record_count)
  databricks_dup_record_count=str(databricks_dup_record_count)
  sap_dup_record_count=str(sap_dup_record_count)
  end_time=datetime.now()
  end_date=str(end_time)
  end_date=end_date[0:19]
  execution_time=str(end_time-start_time)
  execution_time=execution_time[5:]

  if duplicate_record_val_status=='True':
    spark.sql(
          f"""
              INSERT INTO 
                  hive_metastore.generic.tbl_generic_testing_results 
                  (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                  VALUES
                  ('{databricks_view_name}','{SAP_View_Name}','Duplicate Record Count','Duplicate record count validation of databricks and SAP view','{start_date}','{end_date}','{execution_time}','{databricks_dup_record_count}','{sap_dup_record_count}','{diff_dup_count}','','PASS','')
          """
      )
  else:
    spark.sql(
          f"""
              INSERT INTO 
                  hive_metastore.generic.tbl_generic_testing_results 
                  (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                  VALUES
                  ('{databricks_view_name}','{SAP_View_Name}','Duplicate Record Count','Duplicate record count validation of databricks and SAP view','{start_date}','{end_date}','{execution_time}','{databricks_dup_record_count}','{sap_dup_record_count}','{diff_dup_count}','','FAIL','')
          """
      )
  return duplicate_record_val_status

In [0]:
duplicate_record_val_status=Count_duplicate_records(databricks_view_df,sap_dup_record_count)

In [0]:
# Null value count Validation
from pyspark.sql.functions import isnan, when, count, sum
def null_value_counter(databricks_view_df,sap_col_name,sap_col_null_value_count):
  start_time=datetime.now()
  start_date=str(start_time)
  start_date=start_date[0:19]

  from pyspark.sql.types import StringType 
  spark_df = databricks_view_df.select([col(c).cast(StringType()).alias(c) for c in sap_col_name])
  spark_df=spark_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in sap_col_name])
  val=spark_df.collect()[0]
  databricks_col_null_val=[c for c in val]

  sap_col_name_str=""
  if len(sap_col_name)>=1:
    sap_col_name_str=sap_col_name[0]
    if len(sap_col_name)>1:
      for i in range(1,len(sap_col_name)):
        sap_col_name_str=sap_col_name_str+', '+sap_col_name[i]      
  
  col_null_value_count_fail_name=""
  for i in range(len(sap_col_null_value_count)):
    if sap_col_null_value_count[i]==databricks_col_null_val[i]:
      col_null_value_count_validate_status='True'
    else:
      col_null_value_count_validate_status='False'
      databricks_count=str(databricks_col_null_val[i])
      sap_count=str(sap_col_null_value_count[i])
      diff_count=str(abs(databricks_col_null_val[i]-sap_col_null_value_count[i]))
      col_null_value_count_fail_name=sap_col_name[i]
      break

  end_time=datetime.now()
  end_date=str(end_time)
  end_date=end_date[0:19]
  execution_time=str(end_time-start_time)
  execution_time=execution_time[5:]

  if col_null_value_count_validate_status=='True':
    spark.sql(
          f"""
              INSERT INTO 
                  hive_metastore.generic.tbl_generic_testing_results 
                  (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                  VALUES
                  ('{databricks_view_name}','{SAP_View_Name}','Null Value Count',concat('Null value count on ','{sap_col_name_str}',' columns'),'{start_date}','{end_date}','{execution_time}','','','0','','PASS','')
          """
      )
  else:
    spark.sql(
          f"""
              INSERT INTO 
                  hive_metastore.generic.tbl_generic_testing_results 
                  (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                  VALUES
                  ('{databricks_view_name}','{SAP_View_Name}','Null Value Count',concat('Null value count on ','{sap_col_name_str}',' columns'),'{start_date}','{end_date}','{execution_time}','{databricks_count}','{sap_count}','{diff_count}','','FAIL',concat('{col_null_value_count_fail_name}',' column null count is not matching'))
          """
      )
  return True

In [0]:
if sap_col_name[0]!="NO_COLUMN" :
  if len(sap_col_name)!=0 and len(sap_col_null_value_count)!=0:
    from pyspark.sql.functions import col
    sap_col_name_str=""
    if len(sap_col_name)>=1:
      sap_col_name_str=sap_col_name[0]
      if len(sap_col_name)>1:
        for i in range(1,len(sap_col_name)):
          sap_col_name_str=sap_col_name_str+', '+sap_col_name[i]
    if len(sap_col_name)!=0 and len(sap_col_null_value_count)!=0 and len(sap_col_name)==len (sap_col_null_value_count):
      try:
        null_value_counter(databricks_view_df,sap_col_name,sap_col_null_value_count)
      except:
        start_time=datetime.now()
        start_date=str(start_time)
        start_date=start_date[0:19]

        spark.sql(
              f"""
                  INSERT INTO 
                    hive_metastore.generic.tbl_generic_testing_results 
                    (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                    VALUES
                    ('{databricks_view_name}','{SAP_View_Name}','Null Value Count',concat('Null value count on ','{sap_col_name_str}',' columns'),'{start_date}','','','','','','','FAIL','Error occured while validating null record count on columns')
            """
        )
  else:
      start_time=datetime.now()
      start_date=str(start_time)
      start_date=start_date[0:19]
    
      spark.sql(
            f"""
                INSERT INTO 
                    hive_metastore.generic.tbl_generic_testing_results 
                    (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                    VALUES
                    ('{databricks_view_name}','{SAP_View_Name}','Null Value Count',concat('Null value count on ','{sap_col_name_str}',' columns'),'{start_date}','','','','','','','FAIL','Error occured while validating null record count please check list of parameters')
            """
        )

In [0]:
# Sum of measured columns
sap_col_name_str=sap_column_name_lst[0]
if len(sap_col_name_str)>1:
  for i in range(1,len(sap_column_name_lst)):
    sap_col_name_str=sap_col_name_str+', '+sap_column_name_lst[i]
from datetime import datetime
from pyspark.sql.functions import sum
def sum_of_column(databricks_view_df, sap_column_name_lst, sap_column_sum_lst):
  
  sum_df=databricks_view_df.na.fill(0).select([sum(c) for c in sap_column_name_lst])
  val=sum_df.collect()[0]
  databricks_col_sum_lst=[float(c) for c in val]

  for i in range(0,len(sap_column_name_lst)):
    start_time=datetime.now()
    start_date=str(start_time)
    start_date=start_date[0:19]
    sap_col_name_str=str(sap_column_name_lst[i])
    if abs(sap_column_sum_lst[i])==abs(databricks_col_sum_lst[i]):
      end_time=datetime.now()
      end_date=str(end_time)
      end_date=end_date[0:19]
      execution_time=str(end_time-start_time)
      execution_time=execution_time[5:]
      spark.sql(
          f"""
              INSERT INTO 
                  hive_metastore.generic.tbl_generic_testing_results 
                  (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                  VALUES
                  ('{databricks_view_name}','{SAP_View_Name}','Sum of measured columns',concat('Validating Sum on ','{sap_col_name_str}', ' column'),'{start_date}','{end_date}','{execution_time}','','','','0','PASS','')
          """
      )
    else:
      start_time=datetime.now()
      start_date=str(start_time)
      start_date=start_date[0:19]
      sap_col_name_str=str(sap_column_name_lst[i])
      databricks_sum=str(databricks_col_sum_lst[i])
      sap_count=str(sap_column_sum_lst[i])
      diff_count=str(abs(abs(databricks_col_sum_lst[i])-abs(sap_column_sum_lst[i])))
      sum_fail_col_name=str(sap_column_name_lst[i])
      end_time=datetime.now()
      end_date=str(end_time)
      end_date=end_date[0:19]
      execution_time=str(end_time-start_time)
      execution_time=execution_time[5:]
      spark.sql(
          f"""
              INSERT INTO 
                  hive_metastore.generic.tbl_generic_testing_results 
                  (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                  VALUES
                  ('{databricks_view_name}','{SAP_View_Name}','Sum of measured columns',concat('Validating Sum on ','{sap_col_name_str}', ' column'),'{start_date}','{end_date}','{execution_time}','','','','{diff_count}','FAIL',concat('{sum_fail_col_name}',' column sum is not matching'))
          """
      )

In [0]:
if sap_column_name_lst[0]!="NO_COLUMN":
  if len(sap_column_name_lst)!=0 and len(sap_column_sum_lst)!=0 and len(sap_column_name_lst)==len(sap_column_sum_lst):
    try:
      sum_of_column(databricks_view_df, sap_column_name_lst, sap_column_sum_lst)
    except:
      start_time=datetime.now()
      start_date=str(start_time)
      start_date=start_date[0:19]
      spark.sql(
            f"""
                INSERT INTO 
                    hive_metastore.generic.tbl_generic_testing_results 
                    (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                    VALUES
                    ('{databricks_view_name}','{SAP_View_Name}','Sum of measured columns',concat('Validating Sum on ','{sap_col_name_str}', ' columns'),'{start_date}','','','','','','','FAIL','Error occur while validating sum of measured columns')
            """
        )
  else:
      start_time=datetime.now()
      start_date=str(start_time)
      start_date=start_date[0:19]
      spark.sql(
            f"""
                INSERT INTO 
                    hive_metastore.generic.tbl_generic_testing_results 
                    (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                    VALUES
                    ('{databricks_view_name}','{SAP_View_Name}','Sum of measured columns',concat('Validating Sum on ','{sap_col_name_str}', ' columns'),'{start_date}','','','','','','','FAIL','Error occured please check list of column name and count')
            """
        )

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-1901177538400501>:1
----> 1 if sap_column_name_lst[0]!="NO_COLUMN":
      2   if len(sap_column_name_lst)!=0 and len(sap_column_sum_lst)!=0 and len(sap_column_name_lst)==len(sap_column_sum_lst):
      3     try:

TypeError: 'int' object is not subscriptable

In [0]:
def mig_exec_time(databricks_view_name, SAP_View_Name, databricks_database_name):
  execution_time=100
  for i in range(0,10):
    start_time_1=datetime.now()
    spark.sql(f""" select * from {databricks_database_name}.{databricks_view_name}""")
    end_time_1=datetime.now()
    end_date_1=str(end_time_1)
    end_date_1=end_date_1[0:19]
    start_date_1=str(start_time_1)
    start_date_1=start_date_1[0:19]
    execution_tym=str(end_time_1-start_time_1)
    execution_tym=float(execution_tym[5:])
    if execution_tym<execution_time:
      end_date=end_date_1
      start_date=start_date_1
      execution_time=execution_tym
  execution_time='0'+str(execution_time)

  spark.sql(
      f"""
          INSERT INTO 
              hive_metastore.generic.tbl_generic_testing_results 
              (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
              VALUES
              ('{databricks_view_name}','{SAP_View_Name}','Execution Time of databricks view','Execution time of SELECT(*) FROM databricks view','{start_date}','{end_date}','{execution_time}','','','','','NA','Test Status not required')
      """
  )

In [0]:
mig_exec_time(databricks_view_name, SAP_View_Name, databricks_database_name)


In [0]:
#Execution time of sap view
spark.sql(
          f"""
              INSERT INTO 
                  hive_metastore.generic.tbl_generic_testing_results 
                  (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                  VALUES
                  ('{databricks_view_name}','{SAP_View_Name}','Execution Time of sap view','Execution time of SELECT(*) FROM sap view','','','{execution_time_of_sap_view}','','','','','NA','Test Status not required')
          """
      )

In [0]:
notebook_end_datetime=datetime.now()
notebook_end_date=str(notebook_end_datetime)[0:19]
execution_time=str(notebook_end_datetime- notebook_start_datetime)[2:]
execution_time=str(int(execution_time[0:2])*60+int(execution_time[3:5]))+execution_time[5:]
spark.sql(
          f"""
              INSERT INTO 
                  hive_metastore.generic.tbl_generic_testing_results 
                  (Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Check_Difference_Count_Between_Databricks_And_SAP,Check_Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment) 
                  VALUES
                  ('{databricks_view_name}','{SAP_View_Name}','Total notebook execution time','Total time taken to execute test notebook with all the relevant tests','{notebook_start_date}','{notebook_end_date}','{execution_time}','','','','','NA','Test Status not required')
          """
      )

In [0]:
%sql
select * from hive_metastore.generic.tbl_generic_testing_results
order by Test_ID

Test_ID,Databricks_View_Name,SAP_View_Name,Test_Name,Test_Description,Test_Start_Date,Test_End_Date,Test_Execution_Time_In_Seconds,Databricks_View_Count,SAP_View_Count,Difference_Count_Between_Databricks_And_SAP,Difference_Sum_Between_Databricks_And_SAP,Test_validation_Status,Comment
7,v_open_ord_lead1,CA_OPEN_ORD_LEAD1,Records count on filtered column,"Record count on MATNR, WERKS, LEAD1_OPEN_ORDERS filtered columns",2023-05-31 08:58:45,2023-05-31 08:58:53,07.765166,2,2,0,,PASS,
